In [33]:
# Import modules
import torch
torch.manual_seed(7)

In [34]:
# Create a parallel db generator which removes one record from the original db and keeps the rest of the records
def par_db_generator(db, idx):
    return torch.cat((db[0:idx], db[idx+1:]))

In [35]:
def coinflip():
    answer = 0
    if torch.rand(1)>0.5:
        answer=1
        return answer
    else:
        if torch.rand(1)>0.5:
            answer=1
            return answer
        return answer
            

In [36]:
# Backle up the parallel databases together
# it depends entirely on the length of the first db
def par_dbs(db):
    """This creates an array with the datase columns = len(db-1)
    Since we are removing one record at a time and returning the remaining values
    """
    db_all = []
    par_db = torch.ones(db.shape).type(torch.int)
    for i in range(len(db)):
        decision = coinflip()
        if decision:
            par_db[i] = db[i]
        else:
            par_db[i] = 1 if db[i]==0 else 0
    
    db_all.append(par_db)
        
    return db_all

In [37]:
# Create a function to generate the required data based on a given sample of users or records
def generate_db(rec_no):
    """This function generates the db that is then usd to generate the parallel dbs"""
    db = torch.rand(rec_no)>0.5
    # print('Original DB\n{}'.format(db))
    
    #Generate the entire parallel db
    # My Solution
    pdbs = par_dbs(db)
    
    
    # Lecture Solution
#     first_coin_flip = (torch.rand(len(db))>0.5).float()
#     second_coin_flip = (torch.rand(len(db))>0.5).float()
#     pdbs = db.float()*first_coin_flip + (1-first_coin_flip)*second_coin_flip
    
    return db, pdbs    

In [38]:
def db_query(db):
    return db.float().mean()

In [39]:
# Query all the other paraleel DBs and compare with the full database query
def sensitivity(query, num_entries=1000):
    """
    This function takes the query function as an argument and the number of entries
    It initializes the databse and the parallel databases and then performs query for both
    It then uses the qery result to eveluate the similarity
    """
    
    # Generate the data
    torch.manual_seed(500)
    db, pdbs = generate_db(num_entries)
    
    ful_db_query = query(db)
    print(num_entries,'\n')
    print('Without Noise: ', ful_db_query)
    # print(db, '\n', pdbs)
    max_dist = 0
    for pdb in pdbs:
        pdb_query = query(pdb)*2 - 0.5
        print('With Noise: ', pdb_query)
        
        # Creating an L1 Sensitivity
        
        db_distance = torch.abs(pdb_query - ful_db_query)

        if db_distance > max_dist:
            max_dist = db_distance

    return max_dist

In [40]:

for i in [10, 100, 1000, 10000]:
    print("Sensitivity: ", sensitivity(db_query, i), "\n\n")

10 

Without Noise:  tensor(0.7000)
With Noise:  tensor(0.7000)
Sensitivity:  tensor(5.9605e-08) 


100 

Without Noise:  tensor(0.5900)
With Noise:  tensor(0.4800)
Sensitivity:  tensor(0.1100) 


1000 

Without Noise:  tensor(0.5210)
With Noise:  tensor(0.5400)
Sensitivity:  tensor(0.0190) 


10000 

Without Noise:  tensor(0.5070)
With Noise:  tensor(0.4998)
Sensitivity:  tensor(0.0072) 


